#Junhao Zhang "Freddie" ID# 11356533
## Programming Assignment 2 - Part 2
### Cpts 355 - Fall 2015
### An Interpreter for a Postscript-like Language

## Assigned Sept. 21, 2015

### Assigned Sept. 21, 2015
### Due Monday, Oct. 5, 2015
Continue developing your code in the `sps.ipynb` file. You will need to copy some cells from this notebook into your notebook. I strongly encourage you to save a copy of periodically so you can go back in time if you really mess something up. Using a version control system like git or mercurial would be a good idea!

When you are finished, upload `sps.ipynb` on the course Turnin Page for the Interpreter Assignment Part 2. (Don't 
upload using an upload page for any other assignment. Wait until the upload page for this assignment is ready.
If you think it should be ready and it isn't, [email me](mailto:hauser@eecs.wsu.edu).

The entire interpreter project (Parts 1 and Part 2 together) will count for 10% of your course grade. This second part is worth 80% of that 10%.

### This assignment is to be your own work. Refer to the course academic integrity statement in the syllabus.

## The problem
In this assignment you will write an interpreter in Python for a small PostScript-like language, concentrating on key computational features of the abstract machine, omitting all PS features related to graphics, and using a somewhat-simplified syntax.

The simplified language, SPS, has the following features of PS
* integer constants, e.g. `123`: in python3 there is no practical limit on the size of integers
* boolean constants, `true` and `false` (Note that the boolean constants in python are `True` and `False`)
* name constants, e.g. `/fact`: start with a `/` and letter followed by an arbitrary sequence of letters and numbers
* names to be looked up in the dictionary stack, e.g. `fact`: as for name constants, without the `/`
* code constants: code between matched curly braces `{` ... `}`
* built-in operators on numbers: `add`, `sub`, `mul`, `div`, `eq`, `lt`, `gt`
* built-in operators on boolean values: `and`, `or`, `not`; these take boolean operands only. Anything else is an error.
* built-in sequencing operators: `if`, `ifelse`; make sure that you understand the order of the operands on the stack. Play with ghostscript if necessary to help understand what is happening.
* stack operators: `dup`, `exch`, `pop`
* dictionary creation operator: `dictz`; takes no operands
* dictionary stack manipulation operators: `begin`, `end`. `begin` requires one dictionary operand on the operand stack; `end` has no operands.
* name definition operator: `def`. This requires two operands, a name and a value
* defining (using `def`) and calling functions
* stack printing operator (prints contents of stack without changing it): `stack`
* top-of-stack printing operator (pops the top element of the stack and prints it): `=`


## Requirements for Part 2 (Due Oct. 5)
In Part 2 you will continue building the interpreter, making use of everything you built in Part 1. The pieces needed to complete the interpreter are
* Parsing Postscript code
* Handling of *code arrays*
* Handling the `if` and `ifelse` operators
* Function calling
* Actually interpreting SPS programs

### Parsing 
Parsing is the process by which a program is converted to a data structure that can be further processed by an interpreter or compiler. Our SPS programs are very simple to parse. Essentially all we need to do is convert the continuous input text to a list of *tokens* and convert each token to our chosen representation for it. One way to think of tokens is as the minimum-sized "interesting" chunks of the text of a program. In SPS the tokens are: multidigit numbers with optional negative sign, `true` and `false`, multi-character names (with and without a preceding `/`), and the curly brace characters. We've already decided about how some of these will be represented: numbers as Python integers, names as Python strings, booleans as Python booleans, etc. It turns out that what we want for curly braces is not to represent the characters themselves, but rather to represent things falling between the braces, which we do using *code arrays*. Part of parsing is to identify and represent these code arrays, for which we will use Python lists.

### Uses of code arrays 
Recall that a code array is pushed on the stack as a single unit when it is read from the input. Once a code array is on the stack several things can happen: 
* if it is the `then` or `else` part of an `if` or `ifelse` it is recursively interpreted as part of the evaluation of the `if` or `ifelse`. (We will get to interpreting momentarily).
* if it is the top item on the stack when a `def` is executed, it is stored as the value of the name defined by the `def`.
Finally, if when a name is looked up you find that its value is a code array, the code array is recursively interpreted. 

### Key insight
A key insight is that a complete SPS program is essentially a code array. It doesn't have curly braces around it but it is a chunk of code that needs to be interpreted. This suggests how to proceed: convert the SPS program (a string of text) into an array of tokens and code arrays. Define a Python function `interpret` that takes one of these arrays as input and processes it. The `if` and `ifelse` operators choose zero or one of their operands to *recursively* `interpret`. When a name lookup produce a code array as its result, *recursively* `interpret` it,
thus implementing *Postscript function calls*.

### Parsing revisited
Parsing converts an SPS program in the form a string to a program in the form of a code array. It will work in two stages: first, convert all the string to a list of tokens. And then convert the token list to a code array. The difference between the two will be that in the code array, everything between matching curly braces will be represented as a single element (which itself is a code array). In the process of converting from token list to code array the curly braces will disappear, and the string representations of numbers and booleans will be
converted to Python ints and bools.

#### Tokenizing

In [2]:
# For tokenizing we'll use the re package for Regular Expressions. (copy this to your own sps.ipynb notebook) 
import re

def tokenize(s):
    return re.findall("/?[a-zA-Z][a-zA-Z0-9_]*|[-]?[0-9]+|[}{]+|%.*|[^ \t\n]", s)

In [3]:
# See what tokenize does
print (tokenize(
"""
/fact {
dictz
begin
/n exch def
n 2 lt
{1}
{n -1 add fact n mul }
ifelse
end
}def
5 fact =
"""))


['/fact', '{', 'dictz', 'begin', '/n', 'exch', 'def', 'n', '2', 'lt', '{', '1', '}', '{', 'n', '-1', 'add', 'fact', 'n', 'mul', '}', 'ifelse', 'end', '}', 'def', '5', 'fact', '=']


#### Grouping and converting to Python representations for numbers and booleans
The output of `tokenize` isn't fully suitable because things between matching curly braces are not themselves grouped into a code array. We need to convert the output for the above example to 
```
['/fact', ['dictz', 'begin', '/n', 'exch', 'def', 'n', 2, 'lt', [1], ['n', -1, 'add', 'fact', 'n', 'mul'], 'ifelse', 'end'], 'def', 5, 'fact', '=']
```
Notice how in addition to grouping tokens between curly braces into lists, we've also converted the strings that represent numbers to Python numbers; if there were any booleans, those should have been converted to Python booleans as well.

The main issue in how to convert to a code array is how to group things that fall in between matching 
curly braces. In class on Friday, Sept. 18, we went over a couple of strategies for this. I reproduce here the
code that I showed using a python *iterator*. 
```
# The it argument is an iterator that returns left or right parenthesis characters. 
# The sequence of characters returned by the iterator should represent a string of properly nested 
# parentheses pairs, from which the leading '(' has already been removed. If the 
# parenteses are not properly nested, returns False.
def groupMatching(it):
    res = ['(']
    for c in it:
        if c==')':
            res.append(')')
            return res
        else:
            # note how we use a recursive call to group the inner
            # matching parenthesis string and append it as a whole
            # to the list we are constructing.
            # Also note how we've already seen the leading '(' of this
            # inner group and consumed it from the iterator.
            res.append(groupMatching(it))
    return False

# function to turn a string of properly nested parentheses
# into a list of properly nested lists.
def group(s):
    if s[0]=='(':
        return groupMatching(iter(s[1:]))
    else: return False                  # If it starts with ')' it is not properly nested
```
Here I use an interator constructed from a string, but the `iter` 
function will equally well create an iterator from a list. 

Of course, *your code has to deal with curly braces instead of parentheses* and it must also *deal with
strings that contain tokens in addition to the curly braces.* And don't forget that strings representing
numbers and booleans should be converted to ints and bools at this stage as well. Again, I urge you to
not include the curly brace characters in the resulting code array. The structure of the code array itself
is sufficient for what we will do with it.

To illustrate the above point, consider this modified version of `groupMatching` and `group` which
doesn't put the paren characters into its result. Just the *structure* of the result is sufficient
to show the *structure* of the orginal string.


In [4]:
# The it argument is an iterator that returns left or right parenthesis characters. 
# The sequence of return characters should represent a string of properly nested 
# parentheses pairs, from which the leading '(' has already been removed. If the 
# parenteses are not properly nested, returns False.
def groupMatching(it):
    res = []
    for c in it:
        if c=='}':
            return res
        else:
            # note how we use a recursive call to group the inner
            # matching parenthesis string and append it as a whole
            # to the list we are constructing.
            res.append(groupMatching(it))
    return False

# function to turn a string of properly nested parentheses
# into a list of properly nested lists.
def group(s):
    if s[0]=='{':
        return groupMatching(iter(s[1:]))
    else: return False                  # If it starts with ')' it is not properly nested

def _groups(tokens):
    code = []
    cur = ""
    i = 0
    
    while i < len(tokens):
        x = tokens[i]
        if x == '{':
            cur = cur +'{'
            i = i + 1
            tmp = _groups(tokens[i:])

            if not tmp:
                return False
            elif type(tmp) == tuple:
                i = i + int(tmp[1])
                code.append(tmp[0])
                cur = cur + str(tmp[2])
            else:
                i = i + len(tmp)
                code.append(tmp)
            
        elif x=='}':
            return (code,i, cur+'}')
        else:
            code.append(x)
        i = i + 1

    if len(cur) > 0 and cur[0] == '{':
        if group(cur) == False:
            return False
        
    return code

_groups(tokenize(
"""
/fact {
dictz
begin
/n exch def
n 2 lt
{1}
{n -1 add fact n mul }
ifelse
end
}def
5 fact =
"""))


['/fact',
 ['dictz',
  'begin',
  '/n',
  'exch',
  'def',
  'n',
  '2',
  'lt',
  ['1'],
  ['n', '-1', 'add', 'fact', 'n', 'mul'],
  'ifelse',
  'end'],
 'def',
 '5',
 'fact',
 '=']

#### Your parsing implementation 

In [5]:
# In this cell, write your parsing code; it takes a list of tokens produced by tokenize
# and returns a code array; copy this cell into your sps.ipynb notebook and write the 
# necessary code. Of course you may create additional functions to help you write parse()
#
def parse(tokens):
    tmp = _groups(tokens)
    if not tmp or type(tmp) == tuple:
        return False
    else:
        return tmp

parse(tokenize(
"""
/fact {
dictz
begin
/n exch def
n 2 lt
{1}
{n -1 add fact n mul }
ifelse
end
}def
5 fact =
"""))


['/fact',
 ['dictz',
  'begin',
  '/n',
  'exch',
  'def',
  'n',
  '2',
  'lt',
  ['1'],
  ['n', '-1', 'add', 'fact', 'n', 'mul'],
  'ifelse',
  'end'],
 'def',
 '5',
 'fact',
 '=']

## Your Code Start Here

In [6]:
# The operand stack: define the operand stack and its operations in this notebook cell
opstack = []

# now define functions to push and pop values on the opstack according to your decision about which
# end should be the hot end.

def opPop():
    tmp = opstack[-1]
    del opstack[-1]
    return tmp

def opPush(value):
    if value == "0":
        opstack.append(0)
    elif type(value) == str:
        if value[0] == '/':
            opstack.append(value)
        elif lookup(value) != False:
            opstack.append(lookup(value))
        else:
            print("Error from opPush, for " + str(value))
            errorHandler0()
    elif type(value) == list or type(value) == int:
        opstack.append(value)
    else:
        print("Error from opPush, for " + str(value))
        errorHandler0()
    return

#Handle the error
def errorHandler2(x, b):
    opPush(b)
    opPush(x)
    print("Error input: violate the syntax for those two elements")

def errorHandler1(x):
    opPush(x)
    print("Error input: violate the syntax that one element")

def errorHandler0():
    print("Error input: violate the syntax for zero element")
    
#Check enough elements
def enoughInSt1():
    if len(opstack) > 0:
        return True
    else:
        print("No enought elements! Required 1 but only "+str(len(opstack))+" exist(s)")
        return False
    
def enoughInSt2():
    if len(opstack) > 1:
        return True
    else:
        print("No enought elements! Required 2 but only "+str(len(opstack))+" exist(s)")
        return False
    
#Check the status if it is integer in "str" type
def digitOrMinus(num):
    if type(num) == str or type(num)==int:
        if str(num).isdigit():
            return int(num)
        elif str(num[1:]).isdigit():
            if num[0] == "-":
                return 0 - int(num[1:])
            elif num[0] == "+":
                return int(num[1:])
            else:
                return False
        else:
            return False
    else:
        return False

# Remember that there is a Postscript operator called "pop" so we choose different names for these functions.


In [7]:
# The dictionary stack: define the dictionary stack and its operations in this cell
dictstack = []
# now define functions to push and pop dictionaries on the dictstack, to define name, and to lookup a name

def dictPop():
    del dictstack[-1]
    return

def dictPush():
    dictstack.append(opPop())
    return

def define(name, value):
    tmp = {}
    _name = ""
    if type(name) == str and name[0] == '/':
        _name = name[1:]
    else:
        _name = name
    tmp[_name] = value
    dictstack.append(tmp)
    return

#More functionality
def looklookup(name):
    if type(name) is not int and type(name) is not list:
        if type(name) == str:
            if name != "true" and name!= "false":
                if not digitOrMinus(name):
                        for i in range(len(dictstack) - 1, -1, -1):
                            if name in dictstack[i]:
                                return dictstack[i][name]
                        return False
                return digitOrMinus(name)
            else:
                return name
        return False
    else:
        return name
    
def lookup(name):
    
    if type(looklookup(name)) == int or type(looklookup(name)) == list or looklookup(name) == "true" or looklookup(name) == "false" or looklookup(name) == "--nostringval--":
        return looklookup(name)
    elif type(name) == str and type(looklookup(name)) == str:
        return looklookup(looklookup(name))
    return looklookup(name)
    # return the value associated with name
    # what is your design decision about what to do when there is no definition for name


In [8]:
# Arithmetic operators: define all the arithmetic operators in this cell -- add, sub, mul, div, eq, lt, gt
def add():
    if enoughInSt2():
        x = opPop()
        b = opPop()
        if not lookup(x) or not lookup(b) or type(x) is not int or type(b) is not int:
            errorHandler2(x, b)
            print("Error!")
        else:
            opPush(lookup(x)+lookup(b))

        return

def sub():
    if enoughInSt2():
        x = opPop()
        b = opPop()
        if not lookup(x) or not lookup(b) or type(x) is not int or type(b) is not int:
            errorHandler2(x, b)
            print("Error!")
        else:
            opPush(lookup(b)-lookup(x))

        return

def mul():
    if enoughInSt2():
        x = opPop()
        b = opPop()
        if not lookup(x) or not lookup(b) or type(x) is not int or type(b) is not int:
            errorHandler2(x, b)
            print("Error!")
        else:
            opPush(lookup(x)*lookup(b))

        return

def div():
    if enoughInSt2():
        x = opPop()
        b = opPop()
        if not lookup(x) or not lookup(b) or type(x) is not int or type(b) is not int:
            errorHandler2(x, b)
            print("Error!")
        else:
            if lookup(x) != 0:
                opPush(lookup(b)/lookup(x))
            else:
                errorHandler2(x, b)
        return

def gt():
    if enoughInSt2():
        x = opPop()
        b = opPop()
        if not lookup(x) or not lookup(b) or type(x) is not int or type(b) is not int:
            errorHandler2(x, b)
            print("Error!")
        else:
            if lookup(b) > lookup(x):
                opPush("true")
            else:
                opPush("false")

        return

def lt():
    if enoughInSt2():
        x = opPop()
        b = opPop()
        if not lookup(x) or not lookup(b) or type(x) is not int or type(b) is not int:
            errorHandler2(x, b)
            print("Error!")
        else:
            if lookup(b) < lookup(x):
                opPush("true")
            else:
                opPush("false")
        return

def eq():
    if enoughInSt2():
        x = opPop()
        b = opPop()
        if not lookup(x) or not lookup(b):
            errorHandler2(x, b)
            print("Error!")
        else:
            if lookup(b) == lookup(x):
                opPush("true")
            else:
                opPush("false")

        return

In [9]:
# Boolean operators: define all the boolean operators in this cell -- and, or, not

def psAnd():
    if enoughInSt2():
        x = opPop()
        b = opPop()
        m = lookup(x)
        n = lookup(b)
        
        if not m or not b:
            errorHandler2(x, b)
            
        else:
            if m == "true":
                m = True
            elif m == "false":
                m = False
            else:
                errorHandler2(x, b)
                return

            if n == "true":
                n = True
            elif n == "false":
                n = False
            else:
                errorHandler2(x, b)
                return
            if type(m) is bool and type(n) is bool:
                if m and n:
                    
                    opPush("true")
                else:
                    
                    opPush("false")
            else:
                errorHandler2(x,b)

        return


def psOr():
    if enoughInSt2():
        x = opPop()
        b = opPop()
        m = lookup(x)
        n = lookup(b)
        
        if not m or not b:
            errorHandler2(x, b)
            
        else:
            if not digitOrMinus(m):
                if m == "true":
                    m = True
                elif m == "false":
                    m = False
                else:
                    errorHandler2(x, b)
                    return
            else:
                m = digitOrMinus(m)
            if not digitOrMinus(n):
                if n == "true":
                    n = True
                elif n == "false":
                    n = False
                else:
                    errorHandler2(x, b)
                    return
            else:
                n = digitOrMinus(n)

            if type(m) == bool and type(n) == bool:
                if m or n:
               
                    opPush("true")
                else:
             
                    opPush("false")
            elif type(m) == int and type(n) == int:
                opPush(m+n)
            else:
                print("Unexpected error")
        return

def psNot():
    if enoughInSt2():
        x = opPop()
        m = lookup(x)
        
        if not m:
            errorHandler1(x)
            
        else:
            if not digitOrMinus(m):
                if m == "true":
                    m = True
                elif m == "false":
                    m = False
                else:
                    errorHandler1(x)
                    return
            else:
                m = digitOrMinus(m)

            if type(m) == bool:
                if m:
           
                    opPush("false")
                else:
       
                    opPush("true")
            elif type(m) == int:
                #Accordance to Ghostscript
                opPush(0 - m - 1)
            else:
                print("Unexpected error")

        return

In [10]:
# Define the stack manipulation operators in this cell: dup, exch, pop
def dup():
    if enoughInSt1():
        x = opPop()
        opPush(x)
        opPush(x)
    else:
        errorHandler0()
    return
    
    
def exch():

    if enoughInSt2():
        x = opPop()
        b = opPop()
        opPush(x)
        opPush(b)
    else:
        print("error from exch():")
        errorHandler0()
    return
    

In [11]:
# Define the dictionary manipulation operators in this cell: dictz, begin, end, def
# name the function for the def operator psDef because def is reserved in Python

def dictz():
    if enoughInSt1():
        opPush("/--nostringval--")
    else:
        print("Error from dictz")
        errorHandler0()
    return
    
def begin():
    if enoughInSt1():
        x = opPop()
        tmp2 = x

        if type(tmp2) == str:
            if tmp2 == "/--nostringval--":
                tmp2 = tmp2[1:]
            if tmp2 == "--nostringval--":
                tmp = {}
                tmp[tmp2] = "--nostringval--"
                dictstack.append(tmp)
            else:
                print("Error from begin()")
                errorHandler1(x)
    else:
        print("Error from begin()")
        errorHandler0()
    return
    
def end():
    if not lookup("--nostringval--"):
        print("Error from end()")
        errorHandler0()
    else:
        i = len(dictstack) - 1
        while "--nostringval--" not in dictstack[i]:
            dictPop()
            i -= 1
        dictPop()
            
    return
    
def psDef():
    
    if enoughInSt2():
        x = opPop()
        b = opPop()
        define(b,x)
    else:
        print("Error from psDef()")
        errorHandler2(x,b)
    return
    

In [12]:
# Define the printing operators in this cell: =, stack
# Pick a good name for the code implementing =

#=
def popAndPrint():
    if enoughInSt1():
        print(opPop())
    return

def stack():
    if len(opstack) > 0:
        for i in range(len(opstack) - 1, -1, -1):
            print(opstack[i])
    return

In [13]:
# Define any other operators that I may have forgotten in this cell

def roll():
    if enoughInSt2():
        num2 = opPop()
        num = opPop()
        num2 = num2%num
        if abs(num) <= len(opstack):
            if num2 > 0:
                tmp = opstack[0-num:]
                tmp2 = tmp[0-num2:]
                tmp_init = tmp[0:0-num2]
                tmp_final = tmp2
                tmp_final.extend(tmp_init)
                opstack[0-num:] = tmp_final

            elif num2 < 0:
                tmp = opstack[0-num:]
                tmp2 = tmp[0: 0-num2]
                tmp_init = tmp[0-num2:]
                tmp_final = tmp_init
                tmp_final.extend(tmp2)
                opstack[0-num:] = tmp_final
        else:
            errorHandler2(num2, num)
    else:
        errorHandler2()
    return

def count():
    return len(opstack)

def index():
    if enoughInSt1():
        num = opPop()
        tmp = num + 1
        if len(opstack) >= tmp and len(opstack) > 0:
            opPush(opstack[0 - tmp])
        else:
            errorHandler1(num)
    else:
        print("Error input: violate the syntax for those one element")
    return

def clear():
    opstack[:] = []
    
def psIf():
    if enoughInSt2():
        x = opPop()
        b = opPop()
        if b == "true":
            interpret(x)
        elif b != "false":
            errorHandler(x,b)
            print("Error input: violate the syntax for those two elements")
    else:
        print("Error input: violate the syntax for those two elements")
        
def psElif():
    if enoughInSt2():
        x = opPop()
        b = opPop()
        y = opPop()
        if y == "true":
            interpret(b)
        elif y == "false":
            interpret(x)
        else:
            opPush(y)
            opPush(b)
            opPush(x)
            print("Error input: violate the syntax for those two elements")
    else:
        print("Error input: violate the syntax for those two elements")
        

### interpret
We're now ready to write the `interpret` function. It takes a code array as argument, and changes the state of the operand and dictionary stacks according to what it finds there, doing any output indicated by the SPS program (using the `stack` and `=` operators) along the way. `interpret` may be called recursively from the `if` or `ifelse` operators, or when a name is looked up and its value is a code array.

In [14]:
# Copy this cell to your sps.ipynb and write the necessary code; again write
# auxiliary functions if you need them. This will probably be the largest
# function of the whole project, but it will have a very regular and obvious
# structure if you've followed the plan of the assignment.
def interpret(code): # code is a code array
    theBigBigxX = 0
    if code :
         while theBigBigxX < len(code):
                
                if not lookup(code[theBigBigxX])  or type(code[theBigBigxX]) == list:
                
                    if code[theBigBigxX] == "pop":
                        opPop()
                    elif code[theBigBigxX] == "add":
                        add()
                    elif code[theBigBigxX] == "sub":
                        sub()
                    elif code[theBigBigxX] == "mul":
                        mul()
                    elif code[theBigBigxX] == "div":
                        div()
                    elif code[theBigBigxX] == "gt":
                        gt()
                    elif code[theBigBigxX] == "lt":
                        lt()
                    elif code[theBigBigxX] == "eq":
                        eq()
                    elif code[theBigBigxX] == "and":
                        psAnd()
                    elif code[theBigBigxX] == "or":
                        psOr()
                    elif code[theBigBigxX] == "exch":
                        exch()
                    elif code[theBigBigxX] == "not":
                        psNot()
                    elif code[theBigBigxX] == "dup":
                        dup()
                    elif code[theBigBigxX] == "dictz":
                        dictz()
                    elif code[theBigBigxX] == "begin":
                        begin()
                    elif code[theBigBigxX] == "end":
                        end()
                    elif code[theBigBigxX] == "def":
                        psDef()
                    elif code[theBigBigxX] == "=":
                        popAndPrint()
                    elif code[theBigBigxX] == "stack":
                        stack()
                    elif code[theBigBigxX] == "roll":
                        roll()
                    elif code[theBigBigxX] == "count":
                        count()
                    elif code[theBigBigxX] == "index":
                        index()
                    elif code[theBigBigxX] == "clear":
                        clear()
                    elif code[theBigBigxX] == "if":
                        psIf()
                    elif code[theBigBigxX] == "ifelse":
                        psElif()
                    else:
                        opPush(code[theBigBigxX])
                else:
                    if type(lookup(code[theBigBigxX])) == list:
                        interpret(lookup(code[theBigBigxX]))
                    else:
                        opPush(code[theBigBigxX])
                
                #Uncomment to check :P
                
                #print(opstack)
                #print(dictstack)
                #jj = input("continue?")
         
                theBigBigxX = theBigBigxX + 1
    else:
        print("Error call from main:")
        errorHandler0()
    return

## Good One :)

### interpreter
Finally, we can write the `interpreter` function that treats a string as an SPS program and interprets it.

In [15]:
# Copy this cell to your sps.ipynb
def interpreter(s): # s is a string
    interpret(parse(tokenize(s)))


## Testing 
### First test the parsing
Before even attempting to run you full interpreter, make sure that your parsing is working
correctly. Do you get what you expect as the result of the following:
```
parse(tokenize(
'''
true {1} if
'''
))
```
Make sure that the result contains a python boolean and a python integer.
How about
```
parse(tokenize(
'''
true 
{-1}{1} 
ifelse
'''
))
```
Make sure that there are two nested code arrays.

You should know what the correct result is for the following more complicated example. Is
your code producing the right answer? There's not much point in going on until it is.
```
parse(tokenize(
"""
/fact{
   dictz begin
      /n exch def
         n 2 lt
         { 1}
         {n -1 add fact n mul }
      ifelse
   end
}def
5 fact =
"""
))
```
### Finally, test the full interpreter


In [16]:
# Copy this cell to your sps.ipynb. Add tests of your own, each in its own cell, and discuss how they provide
# evidence that your interpreter is working correctly. 
opstack = []
dictstack = []
interpreter(
"""
/fact {
dictz
begin
/n exch def
n 2 lt
{1}
{n -1 add fact n mul }
ifelse
end
}def
5 fact =
""")

120


## Excellent overall. The algorithm is thorough, well reasoned and well presented.
### Final Score = 100